# Advanced Bash Concepts

## 1. While Loops in Bash Scripts

Welcome back. Let's take a moment to quickly recap everything you've learned so far in this module. We started by looking at some of the most common Linux commands, you learned about redirecting streams, pipes and pipelines, and signaling processes. 

Then you've got your first taste of the Bash programming language including how to create a script, use variables in globs, and perform conditional execution. 

That's a lot of new interesting concepts that will help you get the most out of the systems that you work with but we aren't done yet. Let's dive into some more advanced bash concepts before we wrap up this module. Are you ready? Good. Let's do it. 

Bash provides similar looping structures to Python. We can iterate while a condition is true using a while loop and iterate over a list of elements using a for loop. Although of course, the syntax for these loops is slightly different. As a quick reminder, loops are what makes the computer do repetitive tasks for us, anything from working with a bunch of numbers to processing the contents of a file line by line. Our computer doesn't care how many times we ask it to do what we want, it will keep doing them until it's done. No coffee breaks.

```bash
#!/bin/bash
n=1
while [ $n -le 5 ]; do
  echo "Iteration number $n"
  ((n+=1))
done
```

In this script, we're using the variable N to print messages, counting from one to five. The condition for the while loop uses the same format as a condition for an if block. In this example, we check if the variable N is less than or equal to five using the `-le` operator. The loop itself starts with the `do` keyword and finishes with a `done` keyword. To increment the value of the variable N, we're using a bash construct of double parentheses that lets us do arithmetic operations with our variables. It seems complicated but when you take a step back, it all comes together. Let's execute this and see what happens.

```
$ ./while.sh
Iteration number 1
Iteration number 2
Iteration number 3
Iteration number 4
Iteration number 5
```

So that works but what about making our loop a bit more interesting. When using while loops and bash scripts, it's common to have a loop that retries a command a number of times until it succeeds. This is really useful with commands that use network connections or that access resources that might be locked. These commands can fail for external reasons and they're likely to succeed after a retry or two. To simulate a command that sometimes succeeds or sometimes fails, we have a small Python script that will return an exit value picked at random by a range that we give it. 

```python
#!/usr/bin/env python

import sys
import random

value=random.randint(0,3)
print('Returning: ' + str(value))
sys.exit(value)
```

Do you see what the script is doing? It uses random rand int to generate a value between zero and three, then it prints the selected value and exits with it. Let's check this script out.

```
$ python random-exit.py
Returning: 2

$ python random-exit.py
Returning: 1
```

Cool. So we see that we get a value in the zero to three range. Which value we get will depend on each call, that's okay. We want to simulate a command that sometimes fails and sometimes succeeds. Now, let's have a look at our Bash script that will retry the command.

```bash
#!/bin/bash

n=0
command=$1
while ! $command && [ $n -le 5 ]; do
  sleep $n
  ((n=n+1))
  echo "Retry #$n"
done;
```

This script is a bit more complex than the earlier one but not by much. One interesting difference is that we're getting the value of a command line argument using the $1, this is what we use in Bash to access the first command line argument. In Python, we get the same information using sys.argv[1]. 

So we store the parameter and the variable called command, and then we execute the while loop until either the command succeeds or the end variable reaches a value of five. In other words, if the received command fails, we'll retry up to five times. In the body of the while loop, we first sleep a few seconds, then increment the variable and print the number of free try attempts. 

So why do we call the sleep command? This is no time for rest, the idea here is that if the command we're calling is failing due to CPU usage, network or resource exhaustion, it might make sense to wait a bit before trying again. So the more we try, the more we wait. We need to let our computer catch a breather and recover from whatever is making our command fail. In our simulation, the command fails randomly but this retry script works with any other commands that could fail for a wide range of reasons. 

To try this out, we'll need to call our retry script with the random exit command as a parameter like this.

```
$ ./retry.sh ./random-exit.py 
Returning: 2
Retry #1
Returning: 3
Retry #2
Returning: 3
Retry #3
Returning: 0
```

We can see how our script keeps executing until the command that we passed returns zero which is exactly what we wanted, zero problems. This last example is a real-world use case for while loops in Bash and includes a few more advanced topics. So it can definitely feel complex but don't let that stop you. Re-watch this video as many times as it takes and practice the scripts that we covered. Once you're ready, you can meet me over in the next video where we'll look at for loops in Bash.